In [3]:
import numpy as np
!git clone https://github.com/smcantab/chen5595-fall2020
data = np.loadtxt('./chen5595-fall2020/homework/mgm-hw7.txt')

fatal: destination path 'chen5595-fall2020' already exists and is not an empty directory.


In [6]:
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.cluster import KMeans
import copy
from scipy.stats import multivariate_normal as mvn
class MultivariateGaussianMixture(object):
    def __init__(self,
            n_components,
            max_iter,
            mu=None,
            cov=None,
            pi=None):
        """
        Constructor of mixture of Gaussians
        Parameters
        ----------
        n_components : int
        number of gaussian component
        mu : (n_components, ndim) np.ndarray
        mean parameter of each gaussian component
        cov : (n_components, ndim, ndim) np.ndarray
        variance parameter of each gaussian component
        coef : (n_components,) np.ndarray
        mixing coefficients
        """
        self.n_components = n_components
        self.max_iter = max_iter
        self.mu = mu
        self.cov = cov
        self.pi = pi 
        
    # def _gauss(self, X):
    # returns K Gaussians evaluated on data set of observations X
    
    def _expectation(self, X):
        # performs the E step
        # returns the responsibilities resps
        self.gamma = np.zeros((X.shape[0], self.n_components))
        for k in range(self.n_components):
            self.gamma[:,k] = self.pi[k] * mvn.pdf(X, self.mu[k,:], self.cov[k])
        # normalize across columns to make the sum probabilities to 1
        gamma_norm = np.sum(self.gamma, axis=1)[:,np.newaxis]
        self.gamma = self.gamma/gamma_norm
        return self.gamma
    
    
    def _maximization(self, X):
        # performs the M step
        # updates self.mu, self.cov, self.pi
        '''
        Used the aweights feature to update the covariance
        (see more in https://numpy.org/doc/stable/reference/generated/numpy.cov.html)
        
        made sure gamma was giving the 
        right probabilities by normalizing it and making sure sum(gamma[all rows])=1
        '''
        self.pi = np.mean(self.gamma,axis = 0)
        self.mu = np.dot(self.gamma.T,X)/np.sum(self.gamma,axis = 0)
        new_cov1 = (np.cov(X.T,aweights = self.gamma[:,0],ddof = 0))/np.sum(self.gamma[0],axis=0)
        new_cov2 = (np.cov(X.T,aweights = self.gamma[:,1],ddof = 0))/np.sum(self.gamma[1],axis=0)
        self.cov = np.array((new_cov1,new_cov2))
        return self.pi, self.mu, self.cov
   
    def fit(self, X ):
    # write a `while` statement that performs self._expectation and self.maximization
    # until the parameters stop changing (you can use the function np.allclose)
        for i in range(self.max_iter): 
          mu_old = copy.deepcopy(self.mu)
          self.gamma = self._expectation(X)
          self.pi = self._maximization(X)[0]
          self.mu = self._maximization(X)[1]
          self.cov = self._maximization(X)[2]
          if np.allclose(mu_old,self.mu):
            print('No.of iterations =',i)
            break
      
    def responsibilities(self, X):
         return self._expectation(X)

# Intial Parameter Values 
Kmean = KMeans(n_clusters = 2)
Kmean.fit(data)
prediction = Kmean.predict(data)
centers = Kmean.cluster_centers_ 
Cluster1 = np.array(data[prediction==0]).T
Cluster2 = np.array(data[prediction==1]).T
initial_mu = np.array(([-2,6],[0,9])) #would have used centers, but then realized should use a random starting point
initial_cov = np.array((np.cov(Cluster1),np.cov(Cluster2)))
initial_pi = np.array([0.2,0.5])

#First Iteration
GM = MultivariateGaussianMixture(2,1,initial_mu,initial_cov,initial_pi)
GM.fit(data)

print('means for first iteration=',GM.mu)
print('cov for first iteration=',GM.cov)


#Last Iternation 
GM = MultivariateGaussianMixture(2,200,initial_mu,initial_cov,initial_pi)
GM.fit(data)

print('means for last iteration=',GM.mu)
print('cov for last iteration=',GM.cov)


means for first iteration= [[-0.10744593 -0.12954955]
 [ 6.92795282  7.01491676]]
cov for first iteration= [[[ 0.96211662  0.03829517]
  [ 0.03829517  1.04846494]]

 [[ 0.99157214 -0.04258472]
  [-0.04258472  0.9574764 ]]]
No.of iterations = 2
means for last iteration= [[-0.10745973 -0.12956232]
 [ 6.92798199  7.01491402]]
cov for last iteration= [[[ 0.9620355   0.03821625]
  [ 0.03821625  1.04838887]]

 [[ 0.99157061 -0.04258491]
  [-0.04258491  0.9574768 ]]]
